# Problem Setting

Given `N`, consider a convex N-sided polygon with vertices labelled `A[0], A[i], ..., A[N-1]` in clockwise order.

Suppose you triangulate the polygon into `N-2` triangles. For each triangle, the value of that triangle is the product of the labels of the vertices, and the total score of the triangulation is the sum of these values over all `N-2` triangles in the triangulation.

Return the smallest possible total score that you can achieve with some triangulation of the polygon.

Source: LeetCode [link](https://leetcode.com/contest/weekly-contest-135/problems/minimum-score-triangulation-of-polygon/)

![image](https://user-images.githubusercontent.com/8764683/57565247-2de42000-736f-11e9-9111-6fab23625737.png)


<!-- TEASER_END -->

## Example 1
Input: [1,2,3]

Output: 6

Explanation: The polygon is already triangulated, and the score of the only triangle is 6.

## Example 2

Input: [3,7,4,5]

Output: 144

Explanation: There are two triangulations, with possible scores: 3*7*5 + 4*5*7 = 245, or 3*4*5 + 3*4*7 = 144.  The minimum score is 144.

## Example 3
Input: [1,3,1,4,1,5]

Output: 13

Explanation: The minimum score triangulation has score 1 * 1 * 3 + 1 * 1 * 4 + 1 * 1 * 5 + 1 * 1 * 1 = 13.

# Solution from [Laurant](https://leetcode.com/laurant/)

In [7]:
from typing import List
import math
class Solution:
    def minScoreTriangulation(self, A: List[int]) -> int:
        n = len(A)
        a = A + A
        f = [[0] * (n + n) for i in range(n + n)]
        for i in range(n + n, -1, -1):
            for j in range(i + 2, n + n):
                f[i][j] = math.inf
                for k in range(i + 1, j):
                    f[i][j] = min(f[i][j], f[i][k] + f[k][j] + a[i] * a[j] * a[k])
        return f[0][n - 1]
    

In [8]:
# execute
A = [1,3,1,4,1,5]
s = Solution()
s.minScoreTriangulation(A)


13

## Step-by-step understanding
This problem can be solved by using dynamic programming. Overall flow is as blew:

1. Initial setting 
2. Expand a list `A` to a list `A` + `A`
3. Move 2 points `(i, j)` 
4. Move 3rd pointers `k` which divides the polygon specified by `(i, j)`
5. Calculate minimum score of triangle multiplication 
6. Return the score when `(i, j)=(0, n-1)`, which is the original polygon

![image](https://user-images.githubusercontent.com/8764683/57565780-3d1b9b80-7378-11e9-90cd-75d621e74190.png)


### Initial setting
There are three variables created:
* `n`: length of a list `A`
* `a`: `A` + `A`
* `f`: `2n` by `2n` zeros matrix


In [17]:
n = len(A)
a = A + A
f = [[0] * (n + n) for i in range(n + n)]
print(f'n: {n}\na: {a}\n')
print(f'f: ')
f

n: 6
a: [1, 3, 1, 4, 1, 5, 1, 3, 1, 4, 1, 5]

f: 


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

### Move 2 points
`i` ranges from `2n` to 0 and `j` ranges from `i+2` to `2n`

In example 3, `i` moves from 11 to 0 and `j` moves from `i+2` to 11.


In [86]:
# To visualize `f` 
import pandas as pd
from IPython.display import clear_output, display
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)


def highlight_one(val):
    """
    Takes a scalar and returns a string with
    the css property `'background-color: red'` for 1
    , '' otherwise.
    reference: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
    """
    color = 'red' if val == 1 else ''
    return f'background-color: {color}' 

In [69]:
# Initialize f again 
f = [[0] * (n + n) for i in range(n + n)]

for i in range(n + n, -1, -1):
    for j in range(i + 2, n + n):
        clear_output(wait=True)
        f[i][j] = 1
        display(pd.DataFrame(f).style.applymap(highlight_one))


,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0,1,1,1,1,1,1,1,1,1,1
1,0,0,0,1,1,1,1,1,1,1,1,1
2,0,0,0,0,1,1,1,1,1,1,1,1
3,0,0,0,0,0,1,1,1,1,1,1,1
4,0,0,0,0,0,0,1,1,1,1,1,1
5,0,0,0,0,0,0,0,1,1,1,1,1
6,0,0,0,0,0,0,0,0,1,1,1,1
7,0,0,0,0,0,0,0,0,0,1,1,1
8,0,0,0,0,0,0,0,0,0,0,1,1
9,0,0,0,0,0,0,0,0,0,0,0,1


### Move 3rd pointers `k` which divides the polygon specified by `(i, j)`
The 3rd pointer `k` moves from `i+1` to `j`. For example, when `(i, j)==(7, 10)`, `k` moves 8 to 9.

```python
    for k in range(i + 1, j):  
        print(k)
```

In [64]:
i = 7; j = 10
for k in range(i + 1, j):  
    print(k)


8
9


### Calculate minimum score of triangle multiplication
Basically, for each `(i, j)`, it calculates `f[i][k] + f[k][j] + a[i] * a[j] * a[k]` across `k` and keeps only the minimum values. 

```python
f[i][j] = min(f[i][j], f[i][k] + f[k][j] + a[i] * a[j] * a[k])
```

Let's see concrete examples when `(i, j)==(7, 10)`, `k` moves 8 to 9 as mentioned above. This is the case for rectangle like example 2. `k` is a diagonal point. 
* For `k=8`, `f[7][8] + f[8][10] + a[7] * a[10] * a[8]`.
* For `k=9`, `f[7][9] + f[9][10] + a[7] * a[10] * a[9]`.

When we select `k`, it divides the polygon specified by `(i, j)` into three polygons like below. Then, `f[i][k]`, `f[k][j]` should be calculated already and reuse the results. The last term `a[i] * a[j] * a[k]` is the triangle created by `k`.

<img src="https://user-images.githubusercontent.com/8764683/57566140-fa5cc200-737d-11e9-8463-7c38f32a4166.png" alt="split_" width="500" height="600">

In [91]:
# Initialize f again 
f = [[0] * (n + n) for i in range(n + n)]

for i in range(n + n, -1, -1):
    for j in range(i + 2, n + n):
        f[i][j] = math.inf
        for k in range(i + 1, j):
            clear_output(wait=True)
            f[i][j] = min(f[i][j], f[i][k] + f[k][j] + a[i] * a[j] * a[k])
            display(pd.DataFrame(f).style.background_gradient(cmap=cm, low=-0, high=15))


,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0,3,7,8,13,14,17,18,22,23,28
1,0,0,0,12,7,22,13,22,17,29,22,37
2,0,0,0,0,4,9,10,13,14,18,19,24
3,0,0,0,0,0,20,9,20,13,29,18,38
4,0,0,0,0,0,0,5,8,9,13,14,19
5,0,0,0,0,0,0,0,15,8,27,13,38
6,0,0,0,0,0,0,0,0,3,7,8,13
7,0,0,0,0,0,0,0,0,0,12,7,22
8,0,0,0,0,0,0,0,0,0,0,4,9
9,0,0,0,0,0,0,0,0,0,0,0,20
